# Disease Ranking Prototype

This notebook outlines the code for the prototype disease ranking system. It takes in an input of phenotypes, then gets the diseases that match that set. For each disease, each phenotype tf-idf score is calculated and summed up. This sum is then taken over a disease's total tf-idf scores (calculated in a previous notebook) and used as the final score to rank the diseases. Below is the pseudocode for this process.

The given process is meant to give disease output like a diagnostic tool would when entering symptoms or other known physical traits that a patient may have. 

In [1]:
#get phenotype list input
#get the list of diseases associated with those phenotypes
#for each disease:
    #for each phenotype get the tf-idf score
        #get the tf: product of disease and phenotype weight
        #get idf: log of the total diseases over the associated diseases
    #then add up all of those scores
    #output the ratio (sum of listed phenotype scores over the total)
#do this each- get ranking of diseases

In [1]:
import pandas as pd
from py2neo import Graph
from py2neo import Node
from matplotlib import pyplot as plt
import numpy as np

In [13]:
from py2neo import Database
db = Database("bolt://disease.ncats.io:80")
graph = db.default_graph

In [3]:
#get the total tf-idf lists
diseases = pd.read_csv("GARD_TF_IDF.csv")
phenotypes = pd.read_csv("HP_TF_IDF.csv")

In [4]:
total_d = diseases.shape[0]
total_p = diseases.shape[0]

In [5]:
diseases.head()

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07
1,1,3,49,"['GARD:0000003', 'OMIM:200110', 'ORPHA:920', '...","['ABLEPHARON MACROSTOMIA SYNDROME', 'AMS', 'CO...",0.000101,1.140221e-06
2,2,7,14,"['GARD:0000005', 'OMIM:200100', 'ORPHA:14', 'O...","['ABETALIPOPROTEINEMIA', 'BASSEN KORNZWEIG SYN...",0.000037,1.303226e-07
3,3,11,29,"['GARD:0000007', 'OMIM:102370', 'ORPHA:969', '...","['ACROMICRIC DYSPLASIA', 'ACROMICRIC SKELETAL ...",0.000060,3.964164e-07
4,4,15,9,"['GARD:0000011', 'OMIM:104290']","['ALTERNATING HEMIPLEGIA OF CHILDHOOD', 'ALTER...",0.000023,7.009057e-08


In [6]:
phenotypes.head()

,Unnamed: 0,ID(n),count(s),d.id,n.I_CODE,n.N_Name,Weight,tf_idf_sum
0,0,82727,5,HP:0001994,"['MESH:D005198', 'SNOMEDCT_US:236468006', 'UML...","['RENAL TUBULAR FANCONI SYNDROME', ""'DE TONI-F...",0.000022,2.729152e-08
1,1,98209,48,HP:0001394,"['UMLS:C0023890', 'MESH:D008103', 'SNOMEDCT_US...","['HEPATIC CIRRHOSIS', 'SCAR TISSUE REPLACES HE...",0.000131,1.109996e-06
2,2,59646,207,HP:0000365,"['HP:0008563', 'HP:0001754', 'HP:0001728', 'HP...","['HEARING DEFECT', 'CONGENITAL DEAFNESS', 'HEA...",0.000470,1.979341e-05
3,3,75348,43,HP:0003128,"['HP:0005960', 'HP:0003255', 'UMLS:C0347959', ...","['LACTICACIDOSIS', 'LACTICACIDEMIA', 'HYPERLAC...",0.000137,9.326714e-07
4,4,78847,2,HP:0012465,"['UMLS:C4022891', 'HP:0012465']","['INCREASED IRON CONCENTRATION IN LIVER', 'INC...",0.000020,5.927394e-09


In [7]:
#create a function to get the input
def get_disease_ranks(phen_list):
    #phen_list must be a list
    #for now we'll treat these as HP ids
    
    d_sets = []
    for phen in phen_list:
        q = """match (n:S_HP)--(d:DATA) where d.id = {HP_ID} with n match p =(n)-
                    [:R_rel{name: 'has_phenotype'}]-(g:S_GARD) return ID(g);"""
        d = set(graph.run(q, HP_ID = phen).to_series())
        d_sets.append(d)
    
    ids = set.intersection(*d_sets)
    d_df = diseases.loc[diseases['ID(n)'].isin(ids)].copy()
    ratios = []
    for d in ids:
        d_weight = diseases.loc[diseases['ID(n)'] == d, 'Weight'].iloc[0]
        phen_sum = 0
        for phen in phen_list:
            #get the tf-idf
            p_weight = phenotypes.loc[phenotypes['d.id'] == phen, 'Weight'].iloc[0]
            tf = d_weight*p_weight
            associated = phenotypes.loc[phenotypes['d.id'] == phen, 'count(s)'].iloc[0]
            idf = np.log10(total_d/associated)
            tf_idf = tf*idf
            phen_sum += tf_idf
        ratios.append(phen_sum/ diseases.loc[diseases['ID(n)'] == d, 'tf_idf_sum'].iloc[0])
    d_df["rankings"] = ratios
    return d_df

In [44]:
#function that just gives all disease rankings
#does not quite work- it gives a tf-idf score even if the phenotype does not belong to the disease
def get_disease_ranks2(phen_list):
    d_df = diseases.copy()[994:995]
    ratios = []
    for index, row in diseases[994:995].iterrows():
        d_weight = row["Weight"] 
        print(d_weight)
        phen_sum = 0
        for phen in phen_list:
            #get the tf-idf
            p_weight = phenotypes.loc[phenotypes['d.id'] == phen, 'Weight'].iloc[0]
            print(p_weight)
            tf = d_weight*p_weight
            print(tf)
            associated = phenotypes.loc[phenotypes['d.id'] == phen, 'count(s)'].iloc[0]
            print(associated)
            idf = np.log10(total_d/associated)
            print(idf)
            tf_idf = tf*idf
            print(tf_idf)
            phen_sum += tf_idf
            print(phen_sum)
            print("\n")
        ratios.append(phen_sum/ row["tf_idf_sum"])
    d_df["rankings"] = ratios
    return d_df

## Website considerations
- need to run a query to get the overlapping diseases for the phenotype entry
- connect the neo4j database to web app
- connect python to website

In [ ]:
#initial tests of ranking prototype, entering phenotypes from previously generated dataframe

In [9]:
get_disease_ranks(phenotypes["d.id"][:5])

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.356445


In [8]:
get_disease_ranks(phenotypes["d.id"][:2]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.093517
403,403,2714,31,"['GARD:0002658', 'OMIM:276700', 'ORPHA:882', '...","['TYROSINEMIA TYPE 1', 'TYROSINEMIA TYPE I', '...",0.000080,4.768549e-07,0.046735


In [10]:
get_disease_ranks(phenotypes["d.id"][:1]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
13,13,63,36,"['GARD:0000048', 'OMIM:220110', 'UMLS:C0268237']","['CYTOCHROME C OXIDASE DEFICIENCY', 'COMPLEX 4...",0.000078,8.334314e-07,0.019868
493,493,3248,141,"['GARD:0003295', 'OMIM:309000', 'ORPHA:534', '...","['LOWE OCULOCEREBRORENAL SYNDROME', 'OCRL', 'O...",0.000305,8.922160e-06,0.009929
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.006209
1505,1505,9024,51,"['GARD:0009755', 'OMIM:219800']","['NEPHROPATHIC CYSTINOSIS', 'CTNS', 'LYSOSOMAL...",0.000116,1.103330e-06,0.005574
403,403,2714,31,"['GARD:0002658', 'OMIM:276700', 'ORPHA:882', '...","['TYROSINEMIA TYPE 1', 'TYROSINEMIA TYPE I', '...",0.000080,4.768549e-07,0.002021


In [14]:
get_disease_ranks(["HP:0000365","HP:0001319"]).sort_values(by=["rankings"], ascending = False)

,Unnamed: 0,ID(n),count(s),n.I_CODE,n.N_Name,Weight,tf_idf_sum,rankings
197,197,1184,49,"['GARD:0001019', 'OMIM:229200', 'ORPHA:90354',...","['BRITTLE CORNEA SYNDROME', 'EDS VIB (FORMERLY...",0.000103,1.095049e-06,0.224688
1344,1344,8050,24,"['GARD:0008756', 'OMIM:301835', 'UMLS:C0796028']","['ARTS SYNDROME', 'ARTS', 'X-LINKED FATAL ATAX...",0.000048,3.670663e-07,0.088435
771,771,4696,50,"['GARD:0005061', 'OMIM:272200', 'ORPHA:585', '...","['MULTIPLE SULFATASE DEFICIENCY', 'MSD', 'JUVE...",0.000100,1.401906e-06,0.076881
0,0,1,16,"['GARD:0000001', 'OMIM:603358', 'ORPHA:53693',...","['GRACILE SYNDROME', 'FLNMS', 'FINNISH LACTIC ...",0.000046,1.382186e-07,0.062957
1648,1648,9662,83,"['GARD:0010130', 'OMIM:606232', 'ORPHA:48652',...","['22Q13.3 DELETION SYNDROME', 'PHELAN-MCDERMID...",0.000159,2.693412e-06,0.047504
1864,1864,10667,31,"['GARD:0010708', 'OMIM:608027', 'ORPHA:97249',...","['PONTOCEREBELLAR HYPOPLASIA TYPE 3', 'CEREBEL...",0.000057,4.953627e-07,0.039480
948,948,5777,81,"['GARD:0006213', 'OMIM:123450', 'ORPHA:281', '...","['CRI DU CHAT SYNDROME', 'CAT CRY SYNDROME', '...",0.000158,2.895371e-06,0.036436
